In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

In [3]:
os.listdir('../Data/dataFiles/')

['dinesafe.csv',
 'Yelp_Businesses.csv',
 'dinesafe.xml',
 'df_merged_v2.csv',
 'Yelp_data_w_est_id.csv',
 'Statistical_analysis_data_1.csv',
 'Yelp_data_filt_v2.csv',
 'ON_bus.json',
 'dinesafe_v2.xml',
 'ON_bus_v2.json',
 'establishment_names.csv',
 'Cleaned_Data.csv',
 'df_merged.csv']

In [4]:
dinesafe = pd.read_csv('../Data/dataFiles/dinesafe.csv')

In [5]:
dinesafe.head(2)

,Unnamed: 0,action,amount_fined,court_outcome,establishment_address,establishment_id,establishment_name,establishment_status,establishment_type,infraction_details,...,row_id,severity,insp_year,insp_month,int_lat,int_long,int_long_lat,new_int_long,new_int_lat,new_long_lat
0,90826,NaN,NaN,NaN,258 DUPONT ST,10638017,nosh,Pass,Restaurant,NaN,...,90827,NaN,2018,6,4.367497e+07,7.940652e+07,1.230815e+08,79406520,43674970,123081490
1,85159,Notice to Comply,NaN,,850 MILLWOOD RD,10604429,avoca chocolatier,Pass,Restaurant,Fail to Produce Most Recent Food Safety Inspec...,...,85160,NA - Not Applicable,2018,6,4.370462e+07,7.936655e+07,1.230712e+08,79366552,43704622,123071174


In [6]:
dinesafe['establishment_name'] =dinesafe['establishment_name'].str.lower()
dinesafe['establishment_address'] =dinesafe['establishment_address'].str.lower()

In [7]:
dinesafe['int_lat'] = dinesafe['latitude']*(10**6)
dinesafe['int_lat'] = dinesafe['int_lat'].astype(int)
dinesafe['int_long'] = dinesafe['longitude']*-(10**6)
dinesafe['int_long'] = dinesafe['int_long'].astype(int)

In [8]:
Yelp_data = pd.read_csv('../Data/dataFiles/Yelp_data_filt_v2.csv') 


/home/pasindu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (53,54,60) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
cities_to_remove =['Ajax','Ansnorveldt', 'Anthem', 'Beeton', 'Bolton', 'Bradford',  'Bradford West Gwillimbury',\
                 'Brampton','Bridgeville', 'Brooklin', 'Caledon', 'Caledon East','Canonsburg', 'Chardon','Clarkson','Concord','Coraopolis',  'East Ajax', 'East Gwilimbury', 'Gormley', 'Huntersville',\
                  'Kannapolis', 'Kent', 'King City', 'Kingussie','Kleinburg',  'Laval','Ludwigsburg',\
                  'Malton','Maple', 'Mentor-on-the-Lake', 'Midlothian','Missisauga', 'Mississauaga', 'Mississauga',\
                  'Nobleton', 'Oak Ridges', 'Oakridges', 'Oakville','Outremont','Painesville', 'Port Credit','Rantoul','River Drive Park', 'Russellton','Sainte-Julie','Schomberg','Scottsdale',\
                  'Stouffville','Stoughton', 'Streetsboro', 'Streetsville','Strongsville', 'Stuttgart',\
                 'Thornhil', 'Unionville', 'Vaughn City','WICKLIFFE', 'Waunakee','Whiitby', 'Whitby',\
                  'Whitchurch-Stouffville','Whtiby', 'Woodbridge', 'Woodbridge (Vaughan)','oakville']


In [10]:
Yelp_data_v1 = Yelp_data

In [11]:
Yelp_data_v1['int_lat'] = Yelp_data_v1['latitude']*(10**6)
Yelp_data_v1['int_lat'] = Yelp_data_v1['int_lat'].astype(int)
Yelp_data_v1['int_long'] = Yelp_data_v1['longitude']*-(10**6)
Yelp_data_v1['int_long'] = Yelp_data_v1['int_long'].astype(int)

#### Dinesafe ranges of long/lat

In [12]:
ds_min_lat = np.min(dinesafe['int_lat'])

In [13]:
ds_max_lat = np.max(dinesafe['int_lat'])

In [14]:
ds_min_long = np.min(dinesafe['int_long'])

In [15]:
ds_max_long = np.max(dinesafe['int_long'])

#### Narrow down Yelp Data to contain businesses in Dinesafe Data

In [ ]:
Yelp_data_v3 = Yelp_data_v1.copy()

In [ ]:
Yelp_data_v2 = Yelp_data_v1.copy()

In [ ]:
Yelp_data_v2 = Yelp_data_v2[(Yelp_data_v2['int_long'] >= ds_min_long) & (Yelp_data_v2['int_long'] <= ds_max_long)]

In [ ]:
Yelp_data_v2 =Yelp_data_v2.reset_index()
Yelp_data_v2['est_id'] =''
tol = 1000

### Find the same business in the two datasets, using location and Establishment Name

In [ ]:
inds_yelp =[]
inds_ds =[]
for i in range(0,len(Yelp_data_v2)):
    for  j  in range (0, len(dinesafe)):
        if ( abs(Yelp_data_v2.loc[i, 'int_lat'] - dinesafe.loc[j, 'int_lat'] )< tol and
            abs(Yelp_data_v2.loc[i, 'int_long'] - dinesafe.loc[j, 'int_long'] )<tol) and \
            Yelp_data_v2.loc[i,'name'][:5] == dinesafe.loc[j,'establishment_name'][:5] :
            
            Yelp_data_v2.loc[i,'est_id'] = dinesafe.loc[j,'establishment_id']
            inds_yelp.append(i)
            inds_ds.append(j)
    if (i%1000==0):
        print(i)

In [ ]:
Yelp_data_v2['name'] =Yelp_data_v2['name'].str.lower()
Yelp_data_v2['name'] =Yelp_data_v2['name'].str.strip()
Yelp_data_v2['address'] =Yelp_data_v2['address'].str.strip()
Yelp_data_v2['address'] =Yelp_data_v2['address'].str.lower()

In [ ]:
Yelp_data.shape

In [ ]:
Yelp_data_v2.to_csv('Yelp_data_w_est_id.csv')
'../Data/dataFiles/Yelp_data_filt_v2.csv'

In [ ]:
dinesafe_v2 = dinesafe.set_index('establishment_id')
